In [1]:
import numpy as np

# Load calibration data
calibration_file = 'cam_calib\camera_calibration.npz'
calibration_data = np.load(calibration_file)

# Extract calibration parameters
camera_matrix = calibration_data['camera_matrix']
dist_coeffs = calibration_data['dist_coeffs']
rvecs = calibration_data['rvecs']  # Rotation vectors
tvecs = calibration_data['tvecs']  # Translation vectors

In [51]:
import numpy as np

# Given data
drone_world = np.array([0.66, -56, -15.4])  # Replace x, y, z with the drone's world coordinates
camera_relative = np.array([0, 0, 0.24])
pitch_angle_deg = -25  # Negative for downward tilt
pitch_angle_rad = np.radians(pitch_angle_deg)

# Rotation matrix for pitch
R_x = np.array([
    [1, 0, 0],
    [0, np.cos(pitch_angle_rad), -np.sin(pitch_angle_rad)],
    [0, np.sin(pitch_angle_rad), np.cos(pitch_angle_rad)]
])


print(R_x)
print("")
print(camera_relative)
# Rotate camera position
camera_relative_world = R_x @ camera_relative

# Calculate camera position in world coordinates
camera_world = drone_world + camera_relative_world

print("Camera position in world coordinates:", camera_world)


[[ 1.          0.          0.        ]
 [ 0.          0.99120281  0.13235175]
 [ 0.         -0.13235175  0.99120281]]
asdasdasd
Camera position in world coordinates: [  0.66       -55.96823558 -15.16211133]


In [41]:

def calculate_object_location(bbox, depth_image, intrinsic_matrix, camera_position, camera_orientation):
    """
    Calculate the 3D coordinates of an object in world coordinates.

    Parameters:
        bbox (tuple): Bounding box of the object in the format (x_min, y_min, x_max, y_max).
        depth_image (numpy.ndarray): Depth image with depth values in meters.
        intrinsic_matrix (numpy.ndarray): 3x3 camera intrinsic matrix.
        camera_position (numpy.ndarray): 3x1 vector representing the camera position in world coordinates.
        camera_orientation (numpy.ndarray): 3x3 rotation matrix representing the camera orientation in world coordinates.

    Returns:
        numpy.ndarray: 3D coordinates (x, y, z) of the object in world coordinates.
    """
    x_min, y_min, x_max, y_max = bbox

    # Extract the depth value at the center of the bounding box
    u_center = int((x_min + x_max) / 2)
    v_center = int((y_min + y_max) / 2)
    depth = depth_image[v_center, u_center]

    print(depth)
    if depth <= 0 or depth > 80:
        raise ValueError("Invalid depth value: {}. Ensure depth is between 0 and 80 meters.".format(depth))

    # Camera intrinsic parameters
    fx = intrinsic_matrix[0, 0]
    fy = intrinsic_matrix[1, 1]
    cx = intrinsic_matrix[0, 2]
    cy = intrinsic_matrix[1, 2]

    # Convert pixel coordinates to normalized image coordinates
    x_norm = (u_center - cx) / fx
    y_norm = (v_center - cy) / fy

    # Convert to 3D coordinates in the camera frame
    camera_coords = np.array([x_norm * depth, y_norm * depth, depth])

    # Transform to world coordinates
    world_coords = camera_orientation @ camera_coords + camera_position

    return world_coords

In [55]:
from PIL import Image

bounding_box = [136,259,205, 324]  # Example bounding box
depth_image = Image.open("depth.jpg")   # Example depth image
depth_array = np.array(depth_image)[:,:,0]*80/256 # detected range is between 0 to 80 meter, 0 for nearest depth and 80 for farset depth

intrinsic_matrix = camera_matrix # Example intrinsic matrix
camera_position = np.array([-51.87, 2.49, -10.57])  # Camera position in world coordinates
camera_orientation = np.eye(3)  # Example: identity matrix (no rotation)

# Calculate object location
x, y, z = calculate_object_location(bounding_box, depth_array, intrinsic_matrix, camera_position, camera_orientation)
print(f"Object location in world coordinates: x={x}, y={y}, z={z}")

0.875
Object location in world coordinates: x=-52.63688505263953, y=3.2709880027594482, z=-9.695


In [ ]:
{'ts': 1737812989094138368, 
'position': [0.6599999666213989, -56.0, -15.401533126831055], 
'orientation': <scipy.spatial.transform._rotation.Rotation object at 0x0000029D2280D6B0>, 
'linear_velocity': [3.789719755786791e-07, -2.4927132358243398e-08, -0.00014954849029891193], 
'angular_velocity': [-5.281396522373305e-13, 2.223909518761502e-07, -1.3892886840949359e-08], 
'linear_acceleration': [2.5453998020452673e-08, -1.821736539397989e-09, 0.0], 
'has_collided': 0}

In [32]:
from PIL import Image

# Example bounding box and depth image

  # x_min, y_min, x_max, y_max
camera_position = np.array([0.6599999666213989, -56.0, -15.401533126831055])  # Camera position in world coordinates
depth_image =  Image.open("depth.jpg") 
depth_array = np.array(depth_image)

nearest_distance, farthest_distance, world_coords = compute_object_location(
    depth_image=depth_array,
    bbox=bounding_box,
    camera_matrix=camera_matrix,
    rotation_matrix=rotation_matrix,
    camera_position=camera_position
)

print(f"Nearest Distance: {nearest_distance}")
print(f"Farthest Distance: {farthest_distance}")
print("----------[]")
print(f"World Coordinates: {world_coords}")


----------[]
0.0
0.9375
----------[]
Nearest Distance: 0.0
Farthest Distance: 0.9375
----------[]
World Coordinates: [[-17.96234915 -89.96234915 -22.96234915]
 [-17.89189228 -89.89189228 -22.89189228]
 [-17.875      -89.875      -22.875     ]]


In [ ]:
def load_calibration_data(calibration_file):
    """
    Load camera calibration parameters

    Parameters:
        calibration_file 

    Returns:
        dict: camera_matrix، dist_coeffs، rvecs، tvecs
    """
    calibration = np.load(calibration_file)
    calibration_data = {
        'camera_matrix': calibration['camera_matrix'],
        'dist_coeffs': calibration['dist_coeffs'],
        'rvecs': calibration['rvecs'],
        'tvecs': calibration['tvecs']
    }
    return calibration_data

def compute_camera_world_position(drone_world, camera_relative, pitch_angle_deg):
    """

    Parameters:
        drone_world (numpy.ndarray)
        camera_relative (numpy.ndarray)
        pitch_angle_deg (float)

    Returns:
        numpy.ndarray: drone state in world cordinate 
        numpy.ndarray: camera rotate matrix related to drone
    """
    pitch_angle_rad = np.radians(pitch_angle_deg)

    # Pitch rotation in X direction
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(pitch_angle_rad), -np.sin(pitch_angle_rad)],
        [0, np.sin(pitch_angle_rad), np.cos(pitch_angle_rad)]
    ])

    # convert camera cordinate to world cordinate
    camera_relative_world = R_x @ camera_relative
    camera_world = drone_world + camera_relative_world

    return camera_world, R_x

In [ ]:
def check_for_obstacles(depth_img, threshold=1.0):
    """
    search to near ocl

    Parameters:
        depth_img
        threshold

    Returns:
        bool
    """

    if np.any(depth_img < threshold):
        return True
    else:
        return False

In [ ]:
def detect_objects(model, undistorted_img):
    """
    Parameters:
        model
        undistorted_img

    Returns:
        list of detected objects bbx
    """
    with torch.no_grad():
        detections = model.detect(undistorted_img)
    return detections


In [ ]:
def process_depth_image(depth_image_path, scale_factor=80/256):
    """
    process on depth image like scaling and etc.

    Parameters:
        depth_image_path 
        scale_factor  
    """
    
    depth_image = Image.open(depth_image_path)
    depth_array = np.array(depth_image)[:, :, 0] * scale_factor  # detected range is between 0 to 80 meters
    return depth_array

In [ ]:
def calculate_object_location(bbox, depth_image, intrinsic_matrix, camera_position, camera_orientation):
    """
    calculate object location in world cordinate

    Parameters:
        bbox : (x_min, y_min, x_max, y_max)
        depth_image : depth image that scaled to 0 to 80 meters
        intrinsic_matrix : camera matrix
        camera_position 
        camera_orientation

    Returns:
        numpy.ndarray: 3d cordinate of detected objects
    """
    x_min, y_min, x_max, y_max = bbox

    # extract depth centera of detected object
    u_center = int((x_min + x_max) / 2)
    v_center = int((y_min + y_max) / 2)
    depth = depth_image[v_center, u_center]

    print(f"Depth at center: {depth} meters")
    if depth <= 0 or depth > 80:
        raise ValueError(f"Invalid depth value: {depth}. Ensure depth is between 0 and 80 meters.")

    fx = intrinsic_matrix[0, 0]
    fy = intrinsic_matrix[1, 1]
    cx = intrinsic_matrix[0, 2]
    cy = intrinsic_matrix[1, 2]

    # Normalized pixel cordinate
    x_norm = (u_center - cx) / fx
    y_norm = (v_center - cy) / fy

    camera_coords = np.array([x_norm * depth, y_norm * depth, depth])

    world_coords = camera_orientation @ camera_coords + camera_position

    return world_coords

In [ ]:
state = sim.get_drone_state()

qx, qy, qz, qw = state['qx'], state['qy'], state['qz'], state['qw']
rotation_body = R.from_quat([qx, qy, qz, qw])
R_wb = rotation_body.as_matrix()

camera_orientation = R_wb @ R_bc

world_coords = calculate_object_location(
    bbox=det['bbox'],
    depth_image=depth_array,
    intrinsic_matrix=camera_matrix,
    camera_position=camera_world,
    camera_orientation=camera_orientation
)